In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.tsa.arima_model import ARMA,ARMAResults,ARIMA,ARIMAResults
from statsmodels.tsa.statespace.sarimax import SARIMAX

from pmdarima import auto_arima # for determining ARIMA orders
%matplotlib inline
#ignore harmless warnings
import warnings 
warnings.filterwarnings("ignore")
from sklearn.metrics import mean_absolute_error

In [ ]:
df = pd.read_csv("../csv-files/mid-2017-2019-datetime.csv", index_col=[0], parse_dates=[0])
df.head()

In [ ]:
df_gas = pd.read_csv("../csv-files/gas_data.csv",index_col=[0], parse_dates=[0],encoding = "ISO-8859-1")
df_gas.head()


In [ ]:
df=pd.read_csv("../csv-files/mid-gas-2017-2019.csv",index_col=[0], parse_dates=[0])
df.head()

In [ ]:
df_INDO=pd.read_csv('../csv-files/INDO-2017-2019.csv',index_col=[0],parse_dates=[0])

df_INDO.head(3)

In [ ]:
df=pd.merge(df,df_INDO['INDO'], how='inner', left_index=True, right_index=True)
df.head()

In [ ]:
#Splitting the data:
def split_data(data, split_date):
    return data[data.index <= split_date].copy(), \
           data[data.index >  split_date].copy()

def limit(data, frm, to):
    return data[(data.index>=frm)&(data.index<to)]

In [ ]:
df.sort_index(inplace=True)

In [ ]:
t = df.Price.copy()
t = t.drop(t.index[t.index.duplicated()])
freq_index = pd.date_range(start=t.index[0], end=t.index[-1], freq='30T')
constructed = pd.Series(index=freq_index, name='Price')
constructed.update(t)
constructed.interpolate(inplace=True)


In [ ]:
#for gas series:
g = df.Gas.copy()
g = g.drop(g.index[g.index.duplicated()])
freq_index = pd.date_range(start=g.index[0], end=g.index[-1], freq='30T')
constructed_gas = pd.Series(index=freq_index, name='Gas')
constructed_gas.update(g)
constructed_gas.interpolate(inplace=True)


In [ ]:
#for INDO series:
i = df.INDO.copy()
i = i.drop(i.index[i.index.duplicated()])
freq_index = pd.date_range(start=i.index[0], end=i.index[-1], freq='30T')
constructed_indo = pd.Series(index=freq_index, name='INDO')
constructed_indo.update(i)
constructed_indo.interpolate(inplace=True)

In [ ]:
train = limit(constructed, '2019-01-01', '2019-06-15')
test  = limit(constructed, '2019-06-15', '2019-07-01')
test.shape, train.shape



In [ ]:
train_=df[df.index.isin(train.index).copy()]
train_.shape


In [ ]:
#Train and test for Gas Series:
train_gas = limit(constructed_gas, '2017-01-01', '2019-06-15')
test_gas  = limit(constructed_gas, '2019-06-15', '2019-07-01')
test_gas.shape




In [ ]:
train_indo = limit(constructed_indo, '2017-01-01', '2019-06-15')
test_indo = limit(constructed_indo, '2019-06-15', '2019-07-01')
test_indo.shape


In [ ]:
test_gas=df[df.index.isin(test.index)].copy()
test_gas[['Gas']].shape


In [ ]:
test_indo=df[df.index.isin(test_indo.index)].copy()
test_indo[['INDO']].shape


In [ ]:
df['Price'].plot(figsize=(15, 5), title = 'The UK electricity price time-series')

In [ ]:
df['Gas'].plot(figsize=(15, 5), title = 'Gas Price')

In [ ]:
df['Price'].loc[(df['Price'].index >= '2019-06-01') &
               (df['Price'].index < '2019-07-01')] \
    .plot(figsize=(15, 5), title = 'June 2019')

In [ ]:
#ACF and PACF
from statsmodels.tsa.stattools import acovf,acf,pacf,pacf_yw,pacf_ols
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf

In [ ]:
title = 'Autocorrelation: Half-hourly Electricity Prices'
lags = 48
plot_acf(df['Price'],title=title,lags=lags);
#This plot indicates non-stationary data, as there are a large number of lags before ACF values drop off.

In [ ]:
title='Partial Autocorrelation: Half-hourly Electricity Prices'
lags=48
plot_pacf(df['Price'],title=title,lags=lags);

In [ ]:
#First order differencing:
from statsmodels.tsa.statespace.tools import diff

df['d1'] = diff(df['Price'],k_diff=1)
df['d1'].plot(figsize=(12,5));

In [ ]:
title='PACF: Half Hourly Electricity Prices First Difference'
lags=40
plot_pacf(df['d1'].dropna(),title=title,lags=np.arange(lags));  # be sure to add .dropna() here!

In [ ]:
# full autocorrelation plot, it helps to increase the figure size using matplotlib:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(12,5))

plot_acf(df['Price'],ax=ax);

In [ ]:
df['30-Day-SMA'] = df['Price'].rolling(window=48).mean()
df['30-Day-Std'] = df['Price'].rolling(window=48).std()

df[['Price','30-Day-SMA','30-Day-Std']].plot();

In [ ]:
#Stationary Function
from statsmodels.tsa.stattools import adfuller

def adf_test(series,title=''):
    """
    Pass in a time series and an optional title, returns an ADF report
    """
    print(f'Augmented Dickey-Fuller Test: {title}')
    result = adfuller(series.dropna(),autolag='AIC') # .dropna() handles differenced data
    
    labels = ['ADF test statistic','p-value','# lags used','# observations']
    out = pd.Series(result[0:4],index=labels)

    for key,val in result[4].items():
        out[f'critical value ({key})']=val
        
    print(out.to_string())          # .to_string() removes the line "dtype: float64"
    
    if result[1] <= 0.05:
        print("Strong evidence against the null hypothesis")
        print("Reject the null hypothesis")
        print("Data has no unit root and is stationary")
    else:
        print("Weak evidence against the null hypothesis")
        print("Fail to reject the null hypothesis")
        print("Data has a unit root and is non-stationary")

In [ ]:
adf_test(df['Price'],'Stationarity test')

In [ ]:
auto_arima(df['Price'],error_action='ignore').summary()

In [ ]:
stepwise_fit = auto_arima(df['Price'], start_p=0, start_q=0,
                          max_p=6, max_q=3, m=12,
                          seasonal=False,
                          d=None, trace=True,
                          error_action='ignore',   # we don't want to know if an order does not work
                          suppress_warnings=True,  # we don't want convergence warnings
                          stepwise=True)           # set to stepwise

stepwise_fit.summary()

In [ ]:
#ARMA Model:
model = ARMA(train,order=(5,1))
results = model.fit()
results.summary()

In [ ]:
start=len(train)
end=len(train)+len(test)-1
start
predictions = results.predict(start=start, end=end).rename('ARMA(5,1) Predictions')

In [ ]:
title = 'Half Hourly Price Predictions'
ylabel='Price'
xlabel=''

ax = test.plot(legend=True,figsize=(12,6),title=title)
predictions.plot(legend=True)
ax.autoscale(axis='x',tight=True)
ax.set(xlabel=xlabel, ylabel=ylabel);

In [ ]:
#Model Evaluation
from statsmodels.tools.eval_measures import mse,rmse
from sklearn.metrics import mean_absolute_error

error1 = mse(test, predictions)
error2 = rmse(test, predictions)

def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

print(f'MAE Error: {mean_absolute_error(test, predictions)}'), 
print(f'MAPE Error: {mean_absolute_percentage_error(test, predictions)}')
print(f'MSE Error : {error1:11.10}')
print(f'RMSE Error: {error2:11.10}')
print()


In [ ]:
#ARIMA MODELS
model =ARIMA(train,order=(4,1,3))
results = model.fit()
results.summary()

In [ ]:
start=len(train)
end=len(train)+len(test)-1
predictions_arima = results.predict(start=start, end=end).rename('ARIMA(2,1,1) Predictions')

In [ ]:
from statsmodels.tools.eval_measures import mse,rmse
from sklearn.metrics import mean_absolute_error

error1 = mse(test, predictions_arima)
error2 = rmse(test, predictions_arima)

def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

print(f'MAE Error: {mean_absolute_error(test, predictions_arima)}'), 
print(f'MAPE Error: {mean_absolute_percentage_error(test, predictions_arima)}')
print(f'MSE Error : {error1:11.10}')
print(f'RMSE Error: {error2:11.10}')
print()

In [ ]:
title = 'Half Hourly Price Predictions'
ylabel='Price'
xlabel='' 

ax = test.plot(legend=True,figsize=(12,6),title=title)
predictions_arima.plot(legend=True)
ax.autoscale(axis='x',tight=True)
ax.set(xlabel=xlabel, ylabel=ylabel);

In [ ]:
# Plot predictions against known values
title = 'Real adn Predicted MID'
ylabel='Prices'
xlabel='' 
ax = df['Price'].plot(legend=True,figsize=(12,6),title=title)
predictions.plot(legend=True)
ax.autoscale(axis='x',tight=True)
ax.set(xlabel=xlabel, ylabel=ylabel)
#ax.yaxis.set_major_formatter(formatter);

In [ ]:
model = SARIMAX(train,order=(5,1,1),seasonal_order=(1,0,1,12),
                exog=train_gas,
               enforce_stationarity=False,
               enforce_invertibility=False
               )
results = model.fit()
results.summary()

In [ ]:
train.shape

In [ ]:
# Obtain predicted values
start=len(train)
end=len(train)+len(test)-1
exog_forecast=test_gas[['Gas']]
predictions_sarima = results.predict(start=start, end=end,exog=exog_forecast, dynamic=False, typ='levels').rename('SARIMAX(5,1,1)(1,0,1,48) Predictions')
#predictions_sarima = results.predict(start=start, end=end,  
 #                                    dynamic=False,
  #                                   typ='levels').rename('SARIMAX(5,1,1)(1,0,1,12) Predictions')

In [ ]:
predictions_sarima.shape


In [ ]:
from statsmodels.tools.eval_measures import mse,rmse
from sklearn.metrics import mean_absolute_error

error1 = mse(test, predictions_sarima)
error2 = rmse(test, predictions_sarima)

def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

print(f'MAE Error: {mean_absolute_error(test, predictions_sarima)}'), 
print(f'MAPE Error: {mean_absolute_percentage_error(test, predictions_sarima)}')
print(f'MSE Error : {error1:11.10}')
print(f'RMSE Error: {error2:11.10}')
print()

In [ ]:
title = 'Half Hourly Price Predictions'
ylabel='Price'
xlabel='' 

ax = train.plot(legend=True,figsize=(12,6),title=title)
predictions_sarima.plot(legend=True)
ax.autoscale(axis='x',tight=True)
ax.set(xlabel=xlabel, ylabel=ylabel);

In [ ]:
title = 'Half Hourly Price Predictions'
ylabel='Price'
xlabel='' 
ax = test.rename('Actual electricity prices from test set').plot(legend=True,figsize=(12,6),title=title)
predictions_sarima.plot(legend=True)
ax.autoscale(axis='x',tight=True)
ax.set(xlabel=xlabel, ylabel=ylabel);

In [ ]:
#SARIMA with INDO:
model = SARIMAX(train.index,order=(5,1,1),seasonal_order=(1,0,1,48),
                exog=train_indo,
               enforce_stationarity=False,
               enforce_invertibility=False
               )
results = model.fit()
results.summary()

In [ ]:
# Obtain predicted values
start=len(train)
end=len(train)+len(test)-1
exog_forecast=test_indo[['INDO']]
predictions_sarima = results.predict(start=start, end=end,exog=exog_forecast, dynamic=False, typ='levels').rename('SARIMAX(5,1,1)(1,0,1,48) Predictions')

In [ ]:
from statsmodels.tools.eval_measures import mse,rmse
from sklearn.metrics import mean_absolute_error

error1 = mse(test, predictions_sarima)
error2 = rmse(test, predictions_sarima)

def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

print(f'MAE Error: {mean_absolute_error(test, predictions_sarima)}'), 
print(f'MAPE Error: {mean_absolute_percentage_error(test, predictions_sarima)}')
print(f'MSE Error : {error1:11.10}')
print(f'RMSE Error: {error2:11.10}')
print()

In [ ]:
title = 'Half Hourly Price Predictions'
ylabel='Price'
xlabel='' 
ax = test.rename('Actual electricity prices from test set').plot(legend=True,figsize=(12,6),title=title)
predictions_sarima.plot(legend=True)
ax.autoscale(axis='x',tight=True)
ax.set(xlabel=xlabel, ylabel=ylabel);